### Setup

In [1]:
# import pandas
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf

# exchange codes http://finabase.blogspot.com/2014/09/interantional-stock-exchange-codes-for.html

path = './Compustat data3.csv.nosync.csv'
df = pd.read_csv(path, sep=',')

/Users/vini2001/miniforge3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,10,11,12,16,17,22,26,30,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,11

### Configurations

In [61]:
year_start = 2021
year_end = 2022
min_tickers_to_select_row = 4000
only_tickers_with_fiscal_year_ending_at = 12

### Code


#### Remove some exchanges and drop rows without market value

In [62]:
print(len(df))
# drop all df rows where exchg is [0, 1, 2, 19]
df = df[(df['exchg'] != 0) & (df['exchg'] != 1) & (df['exchg'] != 2)]

# # OTCs - it might be better to keep them, because if we exclude them, we will be selecting them if they were OTC at year_start but are not anymore, since compustat updates the exchg for each year with the current value
#                                   So if we use the old values not listed as OTC only, we'll be selecting only the OTCs with great performance, which would be misleading
df = df[(df['exchg'] != 13) & (df['exchg'] != 19)]

# canada
df = df[(df['exchg'] != 7) & (df['exchg'] != 8) & (df['exchg'] != 9)]

# drop rows without market value
df = df[df['mkvalt'].notna()]
print(len(df))

# tickers_to_exclude = ['NE', 'EVTL', 'VAL', 'CHK', 'GPOR', 'FYBR', 'CBL', 'TUEM', 'DTLA.P']
# df = df[~df['tic'].isin(tickers_to_exclude)]
# print(len(df))

211722
211722


#### Aux functions

In [63]:

path_items_descriptions = './items_descriptions.csv'
df_items_descriptions = pd.read_csv(path_items_descriptions, sep=',')
def get_description(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['ItemName'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['ItemDesc']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def get_file_tag(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['FileTag'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['FileTag']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def val_for_tic_at_year(tic, year, month=None):
    msft = yf.Ticker(tic)
    for i in range(0, 5):
        try:
            hist = msft.history(period="max")
            hist.to_csv('hist.csv')
            hist = pd.read_csv('hist.csv')
            break
        except:
            print(f'error getting hist for {tic} {year} {month}')
            continue
        
    dft = df[(df['tic'] == tic) & (df['fyear'] == year)]
    date = dft['datadate'].values[0]
    if month != None : 
        month += 1
        monthN = month if month <= 12 else month - 12
        date = f'{year if month <= 12 else year+1}{monthN if monthN > 9 else f"0{monthN}" }15'

    try:
        date = pd.to_datetime(date, format='%Y%m%d')
    except:
        date = pd.to_datetime(date, format='%Y-%m-%d')
    close = None
    for i in range(0, 5):
        dateStr = date.strftime('%Y-%m-%d')
        valsDay = hist[hist['Date'] == dateStr]
        if len(valsDay) > 0:
            close = valsDay['Close'].values[0]
            break
        else:
            date = date + pd.DateOffset(days=1)
    return close

def market_caps_for_year(year):
    dfyear = df[df['fyear'] == year]
    # return df with mkvalt and tic
    dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
    dfyear = dfyear[['tic', 'mkvalt', 'price', 'ajex', 'prcc_f']]
    #drop nans
    dfyear = dfyear.dropna()
    return dfyear



#### Most frequent columns

In [64]:
columns = df.columns.tolist()
print(columns)

df.head()


tickers = df['tic']
# get set of unique tickers
tickers = set(tickers)
print(f'{len(tickers)} tickers')

path = 'tickers.csv'
df_tickers = pd.DataFrame(tickers)
df_tickers.to_csv(path, index=False)

df['fyear']

['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex', 'ajp', 'bspr', 'compst', 'curcd', 'curncd', 'currtr', 'curuscn', 'final', 'fyr', 'ismod', 'ltcm', 'ogm', 'pddur', 'scf', 'src', 'stalt', 'udpl', 'upd', 'apdedate', 'fdate', 'pdate', 'acchg', 'acco', 'accrt', 'acdo', 'aco', 'acodo', 'acominc', 'acox', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan', 'acqinvt', 'acqlntal', 'acqniintc', 'acqppe', 'acqsc', 'act', 'adpac', 'aedi', 'afudcc', 'afudci', 'aldo', 'am', 'amc', 'amdc', 'amgw', 'ano', 'ao', 'aocidergl', 'aociother', 'aocipen', 'aocisecgl', 'aodo', 'aol2', 'aoloch', 'aox', 'ap', 'apalch', 'apb', 'apc', 'apofs', 'aqa', 'aqc', 'aqd', 'aqeps', 'aqi', 'aqp', 'aqpl1', 'aqs', 'arb', 'arc', 'arce', 'arced', 'arceeps', 'artfs', 'at', 'aul3', 'autxr', 'balr', 'banlr', 'bast', 'bastr', 'batr', 'bcef', 'bclr', 'bcltbl', 'bcnlr', 'bcrbl', 'bct', 'bctbl', 'bctr', 'billexce', 'bkvlps', 'bltb

105       1998.0
106       1999.0
107       2000.0
108       2000.0
109       2001.0
           ...  
868209    2021.0
868216    2021.0
868217    2021.0
868218    2022.0
868219    2022.0
Name: fyear, Length: 211722, dtype: float64

In [65]:

def most_frequent_columns(df):
    map_count = {}
    for c in columns:
        map_count[c] = df[c].describe()['count']

    # sort by count
    map_count = {k: v for k, v in sorted(map_count.items(), key=lambda item: item[1], reverse=True)}

    columns_data = []
    for k, v in map_count.items():
        columns_data.append([k, v, get_description(k), get_file_tag(k)])

    df_columns = pd.DataFrame(columns_data, columns=['Column', 'Count', 'Description', 'FileTag'])
    return df_columns

df_columns = most_frequent_columns(df)

path = './columns.csv'
df_columns.to_csv(path, index=False)
df_columns.head()

,Column,Count,Description,FileTag
0,gvkey,211722.0,Global Company Key - Key Selection,Multiple Files
1,datadate,211722.0,Data Date,Multiple Files
2,fyear,211722.0,Data Year - Fiscal,DES_ANN
3,indfmt,211722.0,Not found,Not found
4,consol,211722.0,Level of Consolidation,CCM_FILEDATE


#### Select rows for year_start

In [66]:
rows_start = df[df['fyear'] == year_start]
rows_start = rows_start.reset_index(drop=True)

# remove this line if datadate already comes as int
# rows_start['datadate'] = rows_start['datadate'].str.replace('-', '').astype(int)

rows_start = rows_start[rows_start['datadate'] <= int(f'{year_start if only_tickers_with_fiscal_year_ending_at <= 12 else year_start+1}{only_tickers_with_fiscal_year_ending_at}31')]

tickers_start = rows_start['tic']
tickers_start = set(tickers_start)
print(f'{len(tickers_start)} tickers in {year_start}')

df_columns_start = most_frequent_columns(rows_start)
path = f'./columns_{year_start}.csv'
df_columns_start.to_csv(path, index=False)
df_columns_start

4353 tickers in 2021


,Column,Count,Description,FileTag
0,gvkey,8909.0,Global Company Key - Key Selection,Multiple Files
1,datadate,8909.0,Data Date,Multiple Files
2,fyear,8909.0,Data Year - Fiscal,DES_ANN
3,indfmt,8909.0,Not found,Not found
4,consol,8909.0,Level of Consolidation,CCM_FILEDATE
...,...,...,...,...
1853,xuwoi_dc,0.0,Underwriting Expense - Other DC,IS_ANN_FS
1854,xuwrei_dc,0.0,Underwriting Expense - Reinsurance DC,IS_ANN_FS
1855,add3,0.0,Address Line 3,COMPANY
1856,add4,0.0,Address Line 4,COMPANY


In [69]:
min_tickers_to_select_row = (2600/4740) * len(rows_start['tic'])
print(min_tickers_to_select_row)
# select columns with count > min_count
def select_columns(df, min_count, file_tag):
    vals = df[df_columns_start['FileTag'] == file_tag]
    vals = vals[vals['Count'] > min_count]['Column']
    return vals.values.tolist()

vals_bal_ann_indl = select_columns(df_columns_start, min_tickers_to_select_row, 'BAL_ANN_INDL')
vals_is_ann_indl = select_columns(df_columns_start, min_tickers_to_select_row, 'IS_ANN_INDL')

print(f'{len(vals_bal_ann_indl)} columns in BAL_ANN_INDL')
print(f'{len(vals_is_ann_indl)} columns in IS_ANN_INDL')

# select all rows in rows_start with columns of vals_bal_ann_indl and vals_is_ann_indl different than NaN
rows_start_bal_and_is = rows_start[['tic', 'fyear'] + vals_bal_ann_indl + vals_is_ann_indl + ['mkvalt', 'prcc_f']]
rows_start_bal_and_is = rows_start_bal_and_is.dropna()
rows_start_bal_and_is = rows_start_bal_and_is.reset_index(drop=True)
# print(len(rows_start_bal_and_is))
print(f'{len(rows_start_bal_and_is)} rows with all columns in {year_start}')

# create csv file
path = f'./rows_{year_start}_bal_ann_indl.csv'
rows_start_bal_and_is.to_csv(path, index=False)

tickers = rows_start_bal_and_is['tic']

data = rows_start_bal_and_is
data

4886.793248945148
16 columns in BAL_ANN_INDL
17 columns in IS_ANN_INDL
3335 rows with all columns in 2021


,tic,fyear,at,dltt,reuna,wcap,emp,icapt,dlc,ao,...,epsfx,epspi,cstke,do,dvp,revt,dvc,dvt,mkvalt,prcc_f
0,AAL,2021.0,66467.000,42181.000,-8638.000,-1670.000,123.400,34841.000,3996.000,5426.000,...,-3.09,-3.09,0.0,0.0,0.0,29882.000,0.000,0.000,11633.1949,17.96
1,CECO,2021.0,416.197,76.522,-36.715,72.326,0.730,282.479,5.217,3.692,...,0.04,0.04,0.0,0.0,0.0,324.140,0.000,0.000,218.2244,6.23
2,ABT,2021.0,75196.000,18252.000,31528.000,11134.000,113.000,54276.000,999.000,4450.000,...,3.94,3.97,-29.0,0.0,0.0,43075.000,3235.000,3235.000,248276.9007,140.74
3,ACU,2021.0,144.439,46.483,69.874,74.976,0.654,123.565,1.389,0.000,...,3.45,3.93,0.0,0.0,0.0,182.088,0.000,0.000,118.6577,33.70
4,AE,2021.0,374.703,14.610,143.040,87.199,0.710,174.996,5.841,3.027,...,2.75,2.78,0.0,0.0,0.0,2025.204,4.112,4.112,121.1126,27.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,IDAI,2021.0,8.665,0.856,-27.208,3.364,0.099,5.890,0.000,0.000,...,-0.48,-0.48,0.0,0.0,0.0,3.678,0.000,0.000,80.7680,4.00
3331,HYFM,2021.0,891.242,158.112,-140.516,180.969,0.720,793.292,9.461,6.453,...,0.31,0.34,0.0,0.0,0.0,479.420,0.000,0.000,1262.2432,28.29
3332,WISH,2021.0,1283.000,16.000,-2545.000,775.000,1.218,834.000,9.000,7.000,...,-0.57,-0.57,0.0,0.0,0.0,2085.000,0.000,0.000,2046.3800,3.11
3333,IBRX,2021.0,468.910,343.417,-1961.921,-34.648,0.587,99.504,302.247,13.976,...,-0.89,-0.89,0.0,0.0,0.0,0.934,0.000,0.000,2418.8064,6.08


#### Regressor

In [82]:
#mlp regressor 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.kernel_ridge import KernelRidge



import numpy as np

# split in x and y
x = data.drop(columns=['tic', 'fyear', 'mkvalt', 'prcc_f'])
y = data['mkvalt']

# normalize x and y
scaler = StandardScaler()
x = scaler.fit_transform(x)
# y = scaler.fit_transform(y.values.reshape(-1, 1))   

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


# MLP Regressor
mlp_regressor = MLPRegressor(
    learning_rate='adaptive',
    max_iter=1000,
    hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100),
    activation='relu',
    solver='adam',
)

mlp_regressor.fit(x_train, y_train)
score = mlp_regressor.score(x_test, y_test)
print(f'MLP Regressor Test Score: {score}')

# mlp_regressor.fit(x, y)
# score = mlp_regressor.score(x, y)
# print(f'MLP Regressor Training Score: {score}')


MLP Regressor Test Score: 0.6545988188379215


: 

#### Simulation

In [75]:
print(f'Analysing {len(data)} tickers')
mapT = {}

market_caps_2021 = market_caps_for_year(year_end)
market_caps_2020 = market_caps_for_year(year_start)

# loop through data
for i in range(len(data)):
    row = data.iloc[i]
    ticker = row['tic']
    x_i = x[i]
    # print(x_i)
    y_i = row['mkvalt']
    # mlp_regressor.predict([x_i])
    p = mlp_regressor.predict([x_i])
    diff = p[0]/y_i - 1
    # print(f'{ticker}: {p} / ${y_i} ({diff})\n')
    if diff > 0:
        mapT[ticker] = diff

# sort mapT by biggest diff
mapT = {k: v for k, v in sorted(mapT.items(), key=lambda item: item[1], reverse=True)}

#get top 30
top30 = list(mapT.keys())[:30]

count = 0
sumAll = 0
sumAllYf = 0
sumAllYfJune = 0
sumAllYfMPlus3 = 0
sumAllYfMPlus3CompletedByMkvalt = 0 # same as sumAllYfMPlus3, but uses compustat data to fill the values when they're not available
noyf = []

for tic in top30:
    vals_2021 = market_caps_2021[market_caps_2021['tic'] == tic]['price'].values
    # we also exclude some tickers here that are presenting a huge change in market cap - they don't have historical prices for 2020 for some reason, doesn't seem like a good idea to use them
    if len(vals_2021) == 0:
        continue
    m2021 = vals_2021[0] 
    m2020 = market_caps_2020[market_caps_2020['tic'] == tic]['price'].values[0] 
    change = m2021/m2020-1
    if np.isnan(change):
        change = (market_caps_2021[market_caps_2021['tic'] == tic]['mkvalt'].values[0] / market_caps_2020[market_caps_2020['tic'] == tic]['mkvalt'].values[0]) - 1
        if np.isnan(change):
            print(f'change is nan for {tic}')
            continue
    exchg = df[df['tic'] == tic]['exchg'].values[0]
    print(f'{tic}: {mapT[tic]} => {change} ({exchg})')
    yf2021 =  val_for_tic_at_year(tic, year_end)
    yf2020 = val_for_tic_at_year(tic, year_start)
    yf2021June = val_for_tic_at_year(tic, year_end, month=18)
    yf2020June = val_for_tic_at_year(tic, year_start, month=18)
    yf2021MPlus3 = val_for_tic_at_year(tic, year_end, month=only_tickers_with_fiscal_year_ending_at+3)
    yf2020MPlus3 = val_for_tic_at_year(tic, year_start, month=only_tickers_with_fiscal_year_ending_at+3)
    if not (yf2021 == None or yf2020 == None):
        changeYf = yf2021/yf2020-1
        sumAllYf += changeYf
        print(f'Yahoo Finance: {changeYf}')
    if not (yf2021June == None or yf2020June == None):
        sumAllYfJune += yf2021June/yf2020June-1
    if not (yf2021MPlus3 == None or yf2020MPlus3 == None):
        sumAllYfMPlus3 += yf2021MPlus3/yf2020MPlus3-1
        sumAllYfMPlus3CompletedByMkvalt += yf2021MPlus3/yf2020MPlus3-1
        print(f'Yahoo Finance M+3: {yf2021MPlus3/yf2020MPlus3-1}')
    else:
        sumAllYfMPlus3CompletedByMkvalt += change
        noyf.append(tic)

    sumAll += change
    count += 1
    print()

change = (sumAll*100)/count
changeYf = (sumAllYf*100)/(count - len(noyf))
changeYfJune = (sumAllYfJune)*100/(count - len(noyf))

print(f'Average change: {change:.2f}%')
print(f'Average change yf: {changeYf:.2f}%')
print(f'Average change yf (Investing only in june after all the fillings have been made public): {changeYfJune:.2f}%')
print(f'Average change yf M+3: {(sumAllYfMPlus3*100)/(count - len(noyf)):.2f}%')
print(f'Average change yf M+3 (Completed by Compustat data): {(sumAllYfMPlus3CompletedByMkvalt*100)/(count):.2f}%')
print(f'Average change yf M+3 (If all delisted went bankrupt): {(sumAllYfMPlus3*100)/(count):.2f}%')
print(f'No yf: {noyf}')
print(f'Count: {count}')
print(f'Coverage: {(count)*100/30:.2f}%')

Analysing 3335 tickers


<ipython-input-63-b4a879c725a5>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
<ipython-input-63-b4a879c725a5>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']


SKYX: 18108.91598085558 => 2519.0 (14)
Yahoo Finance M+3: -0.7079497853354895

ASTI: 743.4443185964178 => -1.0 (14)

ATLX: 679.9728443845556 => 0.19736842105263142 (14)

BRTX: 548.5904933679793 => -0.3663594470046083 (14)
Yahoo Finance: -0.3732719048435228
Yahoo Finance M+3: -0.24123708471784067

CETY: 514.9478273656176 => 126.27272727272727 (14)

CEAD: 510.7377918653905 => -0.8262756172839506 (14)
Yahoo Finance: -0.8166666749450893
Yahoo Finance M+3: -0.5644329877902654

GWAV: 453.457701678017 => -0.9381495744680851 (14)
Yahoo Finance: -0.9361702161837581
Yahoo Finance M+3: -0.875

SBFM: 234.61804114429225 => -0.9452991452991453 (14)
Yahoo Finance M+3: -0.8005780375698064

CELZ: 211.33237390369118 => -0.8275565610859729 (14)
Yahoo Finance: -0.8316742097556997
Yahoo Finance M+3: -0.7377777735392252

FAMI: 175.966615545775 => -0.951852972972973 (14)
Yahoo Finance: -0.9513043476187665
Yahoo Finance M+3: -0.8584615358939538

JXJT: 138.91735558093757 => -0.6371681415929203 (14)
Yahoo Finan